Challenge 4
1. Load the data of the LIGO Hanford detector from [2]. Plot the noise power spectral density (PSD) of that data. Test whether there is any gravitational wave signal present in the data or not? For this test, you can assume the GW signals can produce only from equal mass binary systems with a range between 5 to 10. We assume a threshold on matched filter SNR of 8 to claim detection of GW.
2. Whiten the above data using its noise PSD. You can use PyCBC based function of the Welch method to estimate the PSD. Construct a histogram of the whitened data and show that the whiten data follows a Gaussian distribution with zero mean.


In [1]:
! pip install -q lalsuite
! pip install -q gwpy
! pip install -q pycbc
# -- Click "restart runtime" in the runtime menu
! pip install matplotlib==3.1.3

     |████████████████████████████████| 46.2 MB 1.9 MB/s 
     |████████████████████████████████| 1.4 MB 52.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51 kB 5.3 MB/s 
     |████████████████████████████████| 295 kB 55.2 MB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 3.6 MB 43.5 MB/s 
     |████████████████████████████████| 1.4 MB 5.5 MB/s 
     |████████████████████████████████| 11.2 MB 21.9 MB/s 
     |████████████████████████████████| 895 kB 58.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 6.7 MB 4.4 MB/s 
     |████████████████

The data for this challenge is imported as a timeseries from the google drive. 

In [11]:
from google.colab import drive
from gwpy.timeseries import TimeSeries
import pycbc
from pycbc import frame

drive.mount('/content/drive')

pycbc_strain = pycbc.types.load_timeseries('/content/drive/MyDrive/GW_noisedata-2.npy')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


First we make an arange for the masses in the range between 5 and 10 solar masses increasing by 0.1. We also calculate the sample rate and duration of the data. 

In [18]:
import numpy as ar

m = ar.arange(5,10,0.1)
m1 = m2 = m
print(m)

samplerate = pycbc_strain.sample_rate
duration = pycbc_strain.duration
print('This is the sample rate:', samplerate, 'and the duration:', duration)

[5.  5.1 5.2 5.3 5.4 5.5 5.6 5.7 5.8 5.9 6.  6.1 6.2 6.3 6.4 6.5
 6.6 6.7 6.8 6.9 7.  7.1 7.2 7.3 7.4 7.5 7.6 7.7 7.8 7.9 8.  8.1
 8.2 8.3 8.4 8.5 8.6 8.7 8.8 8.9 9.  9.1 9.2 9.3 9.4 9.5 9.6 9.7
 9.8 9.9]
This is the sample rate: 4096.0 and the duration: 256.0


Here we import all the packeages that we need for the task.

In [19]:
from pycbc.filter import matched_filter
import numpy
from pycbc.waveform import get_td_waveform
import pylab
from pycbc import frame
from pycbc.filter import matched_filter
import numpy
import pylab
from pycbc.filter import resample_to_delta_t, highpass
from pycbc.psd import interpolate, inverse_spectrum_truncation

We then create a template for the cross polarization using the get_td_waveform method and the same approximant as in challenge 2 and 3. However now, we iterate over the different possible masses using a for loop and the previously create arange. 

In [56]:
listsnr = ar.zeros((len(m), 2), float)

for i in range(0, len(m)):
  hp = get_td_waveform(approximant="SEOBNRv4_opt",
                     mass1=m[i],
                     mass2=m[i],
                     delta_t=pycbc_strain.delta_t,
                     f_lower=15)

RuntimeError: ignored

We then follow the same steps to condition the data as in challenge 2 and 3 and create a PSD of the data. 

In [33]:
pycbc_strain = highpass(pycbc_strain, 15.0)
pycbc_strain = resample_to_delta_t(pycbc_strain, 1.0/2048)
conditioned = pycbc_strain.crop(3,3)
# Resize the vector to match our data
hp.resize(len(conditioned))

template = hp.cyclic_time_shift(hp.start_time)


In [41]:
from pycbc.psd import interpolate, inverse_spectrum_truncation
psd = pycbc_strain.psd(4)
# Estimate the power spectral density

# We use 4 second samples of our time series since the data length is 128 which is divisible by 4
psd = conditioned.psd(4)

# Now that we have the psd we need to interpolate it to match our data
# and then limit the filter length of 1 / PSD. After this, we can
# directly use this PSD to filter the data in a controlled manner
psd = interpolate(psd, conditioned.delta_f)

# 1/PSD will now act as a filter with an effective length of 4 seconds
# Since the data has been highpassed above 15 Hz, and will have low values
# below this we need to inform the function to not include frequencies
# below this frequency. 
psd = inverse_spectrum_truncation(psd, int(4 * conditioned.sample_rate),
                                  low_frequency_cutoff=15)

Finally we want to create the SNR plot showing the peaks of SNR for the different masses. THis should be done by using a for loop to iterate over the masses. 

In [42]:
snr = matched_filter(template, conditioned,
                     psd=psd, 
                     low_frequency_cutoff=15
                     )

snr = snr.crop(4 + 4, 4)

# for loop to iterate over masses and find all the maximum SNRs 
# ???

pylab.figure(figsize=[10, 4])
pylab.plot(snr.sample_times, abs(snr))
pylab.ylabel('Signal-to-noise')
pylab.xlabel('Time (s)')
pylab.show()

peak = abs(snr).numpy().argmax()
snrp = snr[peak]
time = snr.sample_times[peak]

print("We found a signal at {}s with SNR {}".format(time, 
                                                    abs(snrp)))

ValueError: ignored

For the second part we first whiten the data as it is done in the tutorial. The we calculate the mean of the whtened data. 

In [43]:
data_whitened = (conditioned.to_frequencyseries() / psd**0.5).to_timeseries()


In [44]:
sum = data_whitened.sum()
mean = sum/len(data_whitened)
print(mean)

-5.9940933682423035e-06


We can see that the mean is approximately zero. Then we want to plot a histogram of the data. The distibution of the data should follow the gaussian distribution around the mean that is equal to (almost) zero.

In [57]:
import matplotlib.pyplot as plot
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

n_bins = 100
plot = plot.hist(data_whitened, n_bins)
plot

(array([1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00,
        1.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        1.00000e+00, 2.40000e+01, 3.74000e+02, 2.32100e+03,
        1.12630e+04, 3.65760e+04, 8.13270e+04, 1.20769e+05,
        1.22110e+05, 8.35310e+04, 3.86210e+04, 1.20780e+04,
        2.58600e+03, 3.64000e+02, 3.50000e+01, 2.00000e+00,
        1.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.0000